In [17]:
import pandas as pd
import numpy as np
import importlib
import gc
import sys
import warnings
sys.path.append('./scripts')  
import preprocesamiento
import feature_engineering
import model_lgb
importlib.reload(preprocesamiento)
importlib.reload(model_lgb)
importlib.reload(feature_engineering)
warnings.filterwarnings("ignore")

# Experimento 7: 
- LGBM
- Usando funcion entrenamiento: semillerio_en_prediccion
- Aumento cantidad de variables
- sqlite:///optuna_studies_v15.db
- Kaggle =  


##### Levantamos el dataset con target ya calculado

In [3]:
df = pd.read_csv("./datasets/periodo_x_producto_con_target.csv", sep=',', encoding='utf-8')
df.shape

(31362, 19)

In [4]:
columnas_baseline = df.columns.tolist()
columnas_baseline

['product_id',
 'periodo',
 'nacimiento_producto',
 'muerte_producto',
 'mes_n',
 'total_meses',
 'producto_nuevo',
 'ciclo_de_vida_inicial',
 'cat1',
 'cat2',
 'cat3',
 'brand',
 'sku_size',
 'stock_final',
 'tn',
 'plan_precios_cuidados',
 'cust_request_qty',
 'cust_request_tn',
 'target']

##### Preprocesamiento a la minima expresión :)

In [50]:
# ##### aplicamos OHE
# df = preprocesamiento.aplicarOHE(df)
# df.shape

### Feature Engineering

##### Neural Prophet

In [5]:
neural_prophet_fe = pd.read_csv("./datasets/features_neuralprophet_completo.csv", sep=',', encoding='utf-8')
neural_prophet_fe['ds'] = pd.to_datetime(neural_prophet_fe['ds'], errors='coerce')
# Versión alternativa más robusta:
neural_prophet_fe['periodo'] = neural_prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
neural_prophet_fe = neural_prophet_fe[['periodo', 'product_id', 'trend', "season_yearly", "season_monthly"]]
df = df.merge(neural_prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 22)

##### Prophet

In [6]:
prophet_fe = pd.read_csv("./datasets/prophet_features_tn_zscore.csv", sep=',', encoding='utf-8')
prophet_fe['ds'] = pd.to_datetime(prophet_fe['ds'], errors='coerce')
prophet_fe['periodo'] = prophet_fe['ds'].apply(
    lambda x: x.year * 100 + x.month if pd.notnull(x) else None
)
prophet_fe = prophet_fe[['periodo', 'product_id', 'trend_add', "yearly_add", "additive_terms", 'trend_mult', 'yearly_mult', 'multiplicative_terms']]
df = df.merge(prophet_fe, on=['periodo', 'product_id'], how='left')
df.shape

(31362, 28)

##### FE Moviles

In [7]:
df = feature_engineering.get_lags(df, "tn", 201912)
df = feature_engineering.get_delta_lags(df, "tn", 24)
df = feature_engineering.get_rolling_means(df, "tn", 201912)
df = feature_engineering.get_rolling_stds(df, "tn", 201912)
df = feature_engineering.get_rolling_mins(df, "tn", 201912)
df = feature_engineering.get_rolling_maxs(df, "tn", 201912)
df = feature_engineering.get_rolling_medians(df, "tn", 201912)
df = feature_engineering.get_rolling_skewness(df, "tn", 201912)
df = feature_engineering.get_autocorrelaciones(df, "tn", 201912)
df.shape

(31362, 590)

In [8]:
df = feature_engineering.get_lags(df, "cust_request_qty", 201912)
df = feature_engineering.get_delta_lags(df, "cust_request_qty", 24)
df = feature_engineering.get_rolling_means(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_stds(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_mins(df, "cust_request_qty", 201912)
df = feature_engineering.get_rolling_maxs(df, "cust_request_qty", 201912)

In [9]:
df = feature_engineering.get_lags(df, "stock_final", 201912)
df = feature_engineering.get_delta_lags(df, "stock_final", 24)
df = feature_engineering.get_rolling_means(df, "stock_final", 201912)
df = feature_engineering.get_rolling_stds(df, "stock_final", 201912)
df = feature_engineering.get_rolling_mins(df, "stock_final", 201912)
df = feature_engineering.get_rolling_maxs(df, "stock_final", 201912)
df.shape

(31362, 1500)

##### FE Moviles sobre otras variables

In [56]:
# #  stock final
# df = feature_engineering.get_lagsEspecificos(df, col='stock_final_zscore')
# df = feature_engineering.get_delta_lags_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_means_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_stds_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_medians_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_mins_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='stock_final_zscore')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='stock_final_zscore')

#  cust_request_qty
# df = feature_engineering.get_lagsEspecificos(df, col='cust_request_qty')
# df = feature_engineering.get_delta_lags_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_means_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_stds_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_mins_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_maxs_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_medians_especificos(df, col='cust_request_qty')
# df = feature_engineering.get_rolling_skewness_especificos(df, col='cust_request_qty')

##### FE Calendario

In [10]:
df = feature_engineering.generar_ids(df)
df = feature_engineering.get_componentesTemporales(df)
df = feature_engineering.get_anomaliasPoliticas(df)
# df = feature_engineering.descomposicion_serie_temporal(df, col='tn')
df.shape

(31362, 1525)

##### FE sobre FE

In [11]:
df = feature_engineering.chatGPT_features_serie(df, "tn")
df = feature_engineering.mes_con_feriado(df)
df.shape

(31362, 1554)

##### Variables Exogenas

In [12]:
df = feature_engineering.get_dolar(df)
df = feature_engineering.get_IPC(df)
df['ipc'] = df['ipc'].str.replace(',', '.').astype(float)
df['dolar'] = df['dolar'].str.replace(',', '.').astype(float)
# df.drop(columns=['ds'], inplace=True)
df.fillna(0, inplace=True) ##### EXPERIMENTAR
df = feature_engineering.correlacion_exogenas(df)
df = feature_engineering.get_mes_receso_escolar(df)
df.shape

(31362, 1557)

##### Nuevas FE

In [13]:
df = feature_engineering.create_ratio_features(df)
df = feature_engineering.enhance_lifecycle_features(df)
df = feature_engineering.create_category_features(df)
df = feature_engineering.create_regime_features(df)
df = feature_engineering.create_nonlinear_trends(df)
df = feature_engineering.create_temporal_interactions(df)
df = feature_engineering.create_asymmetric_window_features(df)
df = feature_engineering.recomendaciones_deepseek(df)
df = feature_engineering.get_nuevas_features(df)
df.shape

(31362, 1599)

##### Elimino aquellas que no sirven

In [ ]:
# importantes = pd.read_csv("./feature_importance/exp04_3.csv", sep=',', encoding='utf-8')
# no_importantes = importantes[importantes['importance'] <= 100]
# no_importantes = no_importantes[~no_importantes['feature'].isin(columnas_baseline)]
# no_importantes

,feature,importance
67,tn_vs_prev_year,100
68,tn_delta_lag5_lag8,99
69,tn_rolling_min_4,96
70,tn_lag_5,95
71,tn_rolling_std_6,95
...,...,...
677,dia_del_year,0
678,cat2_TE,0
680,cat2_PIEL1,0
681,cat2_OTROS,0


In [ ]:
# cols_a_eliminar = no_importantes.feature.unique()
# print(f"Antes de eliminar: {df.shape[1]} columnas")
# df = df.drop(columns=cols_a_eliminar, errors='ignore')
# print(f"Después de eliminar: {df.shape[1]} columnas")

Antes de eliminar: 1144 columnas
Después de eliminar: 684 columnas


Eliminar object/categorical columnas

In [14]:
df = df.select_dtypes(exclude=['datetime', 'datetime64', 'object'])

Train Test Split

In [15]:
train = df[df['periodo'] <= 201912]
test = df[df['periodo'] == 201912]

Entrenamiento

In [16]:
model_lgb.optimizar_con_optuna_con_semillerio_db(train, version="v15")


Para visualizar los resultados en tiempo real:
1. Abre otra terminal y ejecuta:
   optuna-dashboard sqlite:///optuna_studies_v15.db
2. Abre en tu navegador: http://127.0.0.1:8080/


[I 2025-06-30 22:05:36,488] A new study created in RDB with name: lightgbm_optimization_v15
[I 2025-06-30 22:10:45,212] Trial 0 finished with value: 8.484416635084013 and parameters: {'num_leaves': 47, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.892797576724562, 'bagging_fraction': 0.8795975452591109, 'bagging_freq': 2, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'min_child_samples': 45, 'max_depth': 7, 'max_bin': 383, 'min_data_in_leaf': 21, 'extra_trees': True, 'early_stopping_rounds': 18, 'path_smooth': 0.18182496720710062, 'min_gain_to_split': 0.09170225492671691}. Best is trial 0 with value: 8.484416635084013.


Mejor trial hasta ahora: RMSE=8.484417, Parámetros={'num_leaves': 47, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.892797576724562, 'bagging_fraction': 0.8795975452591109, 'bagging_freq': 2, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'min_child_samples': 45, 'max_depth': 7, 'max_bin': 383, 'min_data_in_leaf': 21, 'extra_trees': True, 'early_stopping_rounds': 18, 'path_smooth': 0.18182496720710062, 'min_gain_to_split': 0.09170225492671691}


[I 2025-06-30 22:21:27,633] Trial 1 finished with value: 17.15767005667181 and parameters: {'num_leaves': 41, 'learning_rate': 0.05958389350068958, 'feature_fraction': 0.7727780074568463, 'bagging_fraction': 0.7873687420594125, 'bagging_freq': 7, 'lambda_l1': 1.8007140198129195e-07, 'lambda_l2': 4.258943089524393e-06, 'min_child_samples': 25, 'max_depth': 6, 'max_bin': 414, 'min_data_in_leaf': 36, 'extra_trees': False, 'early_stopping_rounds': 11, 'path_smooth': 0.6075448519014384, 'min_gain_to_split': 0.08526206184364576}. Best is trial 0 with value: 8.484416635084013.


Mejor trial hasta ahora: RMSE=8.484417, Parámetros={'num_leaves': 47, 'learning_rate': 0.2536999076681772, 'feature_fraction': 0.892797576724562, 'bagging_fraction': 0.8795975452591109, 'bagging_freq': 2, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'min_child_samples': 45, 'max_depth': 7, 'max_bin': 383, 'min_data_in_leaf': 21, 'extra_trees': True, 'early_stopping_rounds': 18, 'path_smooth': 0.18182496720710062, 'min_gain_to_split': 0.09170225492671691}


...

[I 2025-07-01 01:01:52,936] Trial 24 finished with value: 6.159770385483371 and parameters: {'num_leaves': 74, 'learning_rate': 0.14568924319434676, 'feature_fraction': 0.9896176725148149, 'bagging_fraction': 0.7858982672977801, 'bagging_freq': 6, 'lambda_l1': 0.05871381583986674, 'lambda_l2': 5.4662403953602196e-05, 'min_child_samples': 46, 'max_depth': 9, 'max_bin': 166, 'min_data_in_leaf': 58, 'extra_trees': True, 'early_stopping_rounds': 44, 'path_smooth': 0.5594887642857059, 'min_gain_to_split': 0.22747772415856038}. Best is trial 14 with value: 1.0106892200633344.


Mejor trial hasta ahora: RMSE=1.010689, Parámetros={'num_leaves': 100, 'learning_rate': 0.11852639795705067, 'feature_fraction': 0.9325299021978121, 'bagging_fraction': 0.9913043905607574, 'bagging_freq': 10, 'lambda_l1': 0.10351163679178885, 'lambda_l2': 1.3129174195141697e-07, 'min_child_samples': 42, 'max_depth': 9, 'max_bin': 220, 'min_data_in_leaf': 78, 'extra_trees': True, 'early_stopping_rounds': 31, 'path_smooth': 0.9652325178791951, 'min_gain_to_split': 0.3734857029985921}


[I 2025-07-01 01:05:55,584] Trial 25 finished with value: 1.5762878385446588 and parameters: {'num_leaves': 90, 'learning_rate': 0.20130980409705235, 'feature_fraction': 0.9016437302675341, 'bagging_fraction': 0.7460290598414079, 'bagging_freq': 8, 'lambda_l1': 0.0011025917499955906, 'lambda_l2': 1.3774897562296066e-07, 'min_child_samples': 36, 'max_depth': 10, 'max_bin': 215, 'min_data_in_leaf': 83, 'extra_trees': True, 'early_stopping_rounds': 25, 'path_smooth': 0.7226957415743933, 'min_gain_to_split': 0.38937888415620175}. Best is trial 14 with value: 1.0106892200633344.


Mejor trial hasta ahora: RMSE=1.010689, Parámetros={'num_leaves': 100, 'learning_rate': 0.11852639795705067, 'feature_fraction': 0.9325299021978121, 'bagging_fraction': 0.9913043905607574, 'bagging_freq': 10, 'lambda_l1': 0.10351163679178885, 'lambda_l2': 1.3129174195141697e-07, 'min_child_samples': 42, 'max_depth': 9, 'max_bin': 220, 'min_data_in_leaf': 78, 'extra_trees': True, 'early_stopping_rounds': 31, 'path_smooth': 0.9652325178791951, 'min_gain_to_split': 0.3734857029985921}
Estudio guardado en: sqlite:///optuna_studies_v15.db

Mejores hiperparámetros encontrados:
num_leaves: 100
learning_rate: 0.11852639795705067
feature_fraction: 0.9325299021978121
bagging_fraction: 0.9913043905607574
bagging_freq: 10
lambda_l1: 0.10351163679178885
lambda_l2: 1.3129174195141697e-07
min_child_samples: 42
max_depth: 9
max_bin: 220
min_data_in_leaf: 78
extra_trees: True
early_stopping_rounds: 31
path_smooth: 0.9652325178791951
min_gain_to_split: 0.3734857029985921


(<optuna.study.study.Study at 0x21580c09510>,
 {'num_leaves': 100,
  'learning_rate': 0.11852639795705067,
  'feature_fraction': 0.9325299021978121,
  'bagging_fraction': 0.9913043905607574,
  'bagging_freq': 10,
  'lambda_l1': 0.10351163679178885,
  'lambda_l2': 1.3129174195141697e-07,
  'min_child_samples': 42,
  'max_depth': 9,
  'max_bin': 220,
  'min_data_in_leaf': 78,
  'extra_trees': True,
  'early_stopping_rounds': 31,
  'path_smooth': 0.9652325178791951,
  'min_gain_to_split': 0.3734857029985921,
  'objective': 'regression',
  'metric': 'rmse',
  'boosting_type': 'gbdt',
  'verbosity': -1})

Prediccion

In [18]:
df_future = model_lgb.semillerio_en_prediccion_con_pesos(train, test, version="v15")

In [19]:
df_future

,periodo,product_id,target,pred
30476,201912,20001,0.0,1312.098207
30477,201912,20002,0.0,1016.446887
30478,201912,20003,0.0,728.742897
30479,201912,20004,0.0,717.139216
30480,201912,20005,0.0,678.526221
...,...,...,...,...
31357,201912,21265,0.0,-1.218135
31358,201912,21266,0.0,-2.318349
31359,201912,21267,0.0,-2.595775
31360,201912,21271,0.0,1.564772


Filtramos los 180 productos

In [20]:
productos_ok = pd.read_csv("https://storage.googleapis.com/open-courses/austral2025-af91/labo3v/product_id_apredecir201912.txt", sep="\t")
df_future = df_future[df_future['periodo'] == 201912]
df_future = df_future[df_future['product_id'].isin(productos_ok['product_id'].unique())]


In [21]:
df_future

,periodo,product_id,target,pred
30476,201912,20001,0.0,1312.098207
30477,201912,20002,0.0,1016.446887
30478,201912,20003,0.0,728.742897
30479,201912,20004,0.0,717.139216
30480,201912,20005,0.0,678.526221
...,...,...,...,...
31355,201912,21263,0.0,1.518800
31357,201912,21265,0.0,-1.218135
31358,201912,21266,0.0,-2.318349
31359,201912,21267,0.0,-2.595775


Vemos cuantos negativos hay

In [22]:
df_future[df_future['pred'] < 0]

,periodo,product_id,target,pred
30549,201912,20079,0.0,-5.693166
30570,201912,20102,0.0,-17.693306
30629,201912,20168,0.0,-1.682108
30643,201912,20183,0.0,-1.710660
30647,201912,20187,0.0,-1.216176
...,...,...,...,...
31344,201912,21233,0.0,-1.073537
31354,201912,21262,0.0,-0.944677
31357,201912,21265,0.0,-1.218135
31358,201912,21266,0.0,-2.318349


Reemplazamos los negativos por el promedio de ultimos 12 meses

In [24]:
promedio780 = model_lgb.promedio_12_meses_780p()
df_future = df_future.merge(promedio780, on='product_id', how='left')
df_future.drop(columns=['target','periodo'], inplace=True)
df_future.loc[df_future['pred'] < 0, 'pred'] = df_future['tn']
df_future



,product_id,pred,tn
0,20001,1312.098207,1454.732720
1,20002,1016.446887,1175.437142
2,20003,728.742897,784.976407
3,20004,717.139216,627.215328
4,20005,678.526221,668.270104
...,...,...,...
775,21263,1.518800,0.029993
776,21265,0.089541,0.089541
777,21266,0.094659,0.094659
778,21267,0.092835,0.092835


Guardamos el archivo

In [25]:
df_future.drop(columns=['tn'], inplace=True)
df_future.rename(columns={'pred': 'tn'}, inplace=True)
df_future.to_csv("./outputs/predicciones_exp_07_lgb_v2.csv", index=False, sep=',')